In [ ]:
from tweet_parser.tweet import Tweet
from tweet_parser.tweet_parser_errors import NotATweetError, UnexpectedFormatError
import fileinput
import json
import string
import re
import time
import csv
import geopy.distance
import nltk
# nltk.download('vader_lexicon')
# nltk.download('universal_tagset')
from nltk.util import ngrams
from nltk.sentiment import SentimentIntensityAnalyzer
from google.cloud import storage

# Create bucket client and authenticate via service account access key
storage_client = storage.Client.from_service_account_json('/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json')

BUCKET_NAME = 'allteamsmlb' # Do not change

sia = SentimentIntensityAnalyzer()

print('El Fin')

In [2]:
team_names = [
    '02Angels', '03Athletics', '04Mariners', '05Astros', '06Rangers', '07Rockies', '08Padres', '09Dodgers', '10Giants', '11Diamondbacks',
    '12Cardinals', '13Reds', '14Cubs', '15Pirates', '16Brewers', '17Braves', '18Mets', '19Phillies', '20Marlins', '21Nationals',
    '22Indians', '23Twins', '24Royals', '25Tigers', '26WhiteSox', '27Yankees', '28Rays', '29RedSox', '30Orioles', '31BlueJays'
]

team_abbrevs = [
    'ANA', 'OAK', 'SEA', 'HOU', 'TEX', 'COL', 'SDP', 'LAD', 'SFG', 'AZ', 'STL', 'CIN', 'CHC', 'PIT', 'MIL',
    'ATL', 'NYM', 'PHI', 'FLA', 'WSN', 'CLE', 'MIN', 'KCR', 'DET', 'CHW', 'NYY', 'TBD', 'BOS', 'BAL', 'TOR'
]

# TODO: Check for negative longitudes in the tweets
team_lats = {
    'ANA': 33.800031, 'OAK': 37.751614, 'SEA': 47.591217, 'HOU': 29.757224, 'TEX': 32.751207,
    'COL': 39.755878, 'SDP': 32.707206, 'LAD': 34.07358, 'SFG': 37.778361, 'AZ': 33.445498,
    'STL': 38.62248, 'CIN': 39.097392, 'CHC': 41.948036, 'PIT': 40.446993, 'MIL': 43.02834,
    'ATL': 33.890961, 'NYM': 40.756743, 'PHI': 39.905763, 'FLA': 25.778194, 'WSN': 38.872705,
    'CLE': 41.496183, 'MIN': 44.981703, 'KCR': 39.051355, 'DET': 42.339308, 'CHW': 41.830087,
    'NYY': 40.829519, 'TBD': 27.768214, 'BOS': 42.346361, 'BAL': 39.283658, 'TOR': 43.641684
}

team_longs = {
    'ANA': -117.883017, 'OAK': -122.200574, 'SEA': -122.332721, 'HOU': -95.35521, 'TEX': -97.082635,
    'COL': -104.994192, 'SDP': -117.15706, 'LAD': -118.240147, 'SFG': -122.389712, 'AZ': -112.066694,
    'STL': -90.193205, 'CIN': -84.506852, 'CHC': -87.65569, 'PIT': -80.005987, 'MIL': -87.971451,
    'ATL': -84.467772, 'NYM': -73.845994, 'PHI': -75.166574, 'FLA': -80.219668, 'WSN': -77.007632,
    'CLE': -81.685699, 'MIN': -93.278072, 'KCR': -94.480666, 'DET': -83.048876, 'CHW': -87.63405,
    'NYY': -73.926739, 'TBD': -82.653295, 'BOS': -71.097631, 'BAL': -76.621801, 'TOR': -79.389235
}

super_list_old = [
    ['NOUN', 'VERB', 'ADV'], ['NOUN', 'VERB', 'ADP'], ['NOUN', 'VERB', 'ADV', 'ADJ'],
    ['NOUN', 'VERB', 'NOUN', 'ADP'], ['NOUN', 'VERB', 'ADJ'], ['NOUN', 'VERB', 'ADJ', 'NOUN']
]

# TODO: Get new super_list
super_list_new = []

# Happy Emoticons
emoticons_happy_old = [
    ':-)', ':)', ';)', ':o)', ':]', ':c)', ':>', '=]', '=)', ':}', ':^)', ':-D',
    ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D', ':-))', ":'-)",
    ":')", ':^*', '>:P', ':-P', ':P', ':-p', ':p', '=p', ':-b', ':b', '>:)',
    '>;)', '>:-)', '<3', '☺️', '😚', '😙', '😋', '😛', '😜', '😝', '🤗', '😏',
    '😌', '🥳', '😎', '😺', '😸', '😹', '😻', '😼', '😽', '👍', '👏', '🙌',
    '😀', '😃', '😄', '😁', '😆', '😅', '🤣', '😂', '🙂', '😉', '😊', '😇',
    '🥰', '😍', '🤩', '😘', '🎉', '🎊', '💘', '💝', '💖', '💗', '💓', '💞',
    '💕', '💟', '❣️', '❤️‍🔥', '❤️', '🧡', '💛', '💚', '💙', '💜', '🤎', '🖤',
    '🤍', '💯'
    ]

emoticons_happy_new = [
    ':-)', ':)', ';)', ':o)', ':]', ':c)', '=]', '=)', ':^)', ':D', 'xD', 'XD',
    '=-D', '=D', ':-))', ":'-)", ":')", '>:)', '>;)', '>:-)', '<3', '☺️', '😚',
    '😙', '😋', '😛', '😜', '😝', '🤗', '😏', '😌', '🥳', '😎', '😺', '😸', '😹', '😻',
    '😼', '😽', '👍', '👏', '🙌', '😀', '😃', '😄', '😁', '😆', '😅', '🤣', '😂', '🙂',
    '😉', '😊', '😇', '🥰', '😍', '🤩', '😘', '🎉', '🎊', '💘', '💝', '💖', '💗', '💓',
    '💞', '💕', '💟', '❣️', '❤️‍🔥', '❤️', '🧡', '💛', '💚', '💙', '💜', '🤎',
    '🖤', '🤍', '💯'
    ]

# Sad Emoticons
emoticons_sad_old = [
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', '>:\\', ';(', '😒', '😔', '🤢', '🤮', '🤧', '🥴', '😕', '😟', '🙁',
    '☹️', '😦', '😧', '😨', '😰', '😥', '😢', '😭', '😱', '😖', '😣', '😞',
    '😓', '😩', '😫', '😤', '😡', '😠', '🤬', '👿', '😿', '😾', '🖕', '👎',
    '🙍', '🙍‍♂️', '🙍‍♀️', '🙎', '🙎‍♂️', '🙎‍♀️', '🤦', '🤦‍♂️', '🤦‍♀️', '💔', '💢'
    ]

# Sad Emoticons
emoticons_sad_new = [
    ':-/', '>:/', '>:[', ':-(', ':[', ':-||', ':-[', ':-<', '=\\', '=/', '>:(',
    ':(', '>.<', ":'-(", ":'(", ':\\', ':-c', ':c', '>:\\', ';(', '😒', '😔', '🤢',
    '🤮', '🤧', '🥴', '😕', '😟', '🙁', '☹️', '😦', '😧', '😨', '😰', '😥', '😢',
    '😭', '😱', '😖', '😣', '😞', '😓', '😩', '😫', '😤', '😡', '😠', '🤬', '👿', '😿',
    '😾', '🖕', '👎', '🙍', '🙍‍♂️', '🙍‍♀️', '🙎', '🙎‍♂️', '🙎‍♀️', '🤦', '🤦‍♂️',
    '🤦‍♀️', '💔', '💢'
    ]

print('El Fin')

El Fin


In [3]:
def clean_tweet(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # remove old style retweet text 'RT'
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags
    tweet = re.sub(r'#', '', tweet)

    return tweet


intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),  # 60 * 60 * 24
    ('hours', 3600),  # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
)


def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(round(value), name))
    return ', '.join(result[:granularity])


def increment_dict(key, dictionary):
    if key in dictionary.keys():
        dictionary[key] += 1
    else:
        dictionary[key] = 1


def write_dict_to_file(file_name, dictionary):
    with open(file_name, 'w') as new_file:
        for key, value in dictionary.items():
            new_file.write('%s : %d\n' % (key, value))

print('El Fin')

El Fin


In [ ]:
# Start runtime
t0 = time.time()


# ALL TEAMS DICTS
ALL_TEAMS_total_bow = {}
ALL_TEAMS_total_tweets = {}


for team_name in team_names:

    team_abbrev = team_abbrevs[team_names.index(team_name)]

    dates = []

    for i in range(16, 20):
        # Atlanta 2016 is in a different stadium. Skip.
        if team_abbrev == 'ATL' and i == 16:
            continue
        with open(('Data/Team_WL/' + str(team_abbrev) + str(i) + '.csv'), newline='', encoding='utf-8-sig') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in reader:
                date = row[0]
                if (date.startswith('d')):
                    continue
                if not date.startswith('1'):
                    date = '0' + date
                if not date[5] == '/':
                    date = date[:3] + '0' + date[3:]
                date = date[6:] + '-' + date[:2] + '-' + date[3:-5]
                dates.append(date)


    # Average distances between tweets and stadium
    COM_average_distance = 0
    IP_average_distance = 0
    NP_average_distance = 0

    # COMBINED DICTS

    COM_total_bow = {}
    COM_total_tweets = {}
    COM_tweets_per_hour = {}

    # DICTS FOR TEAM IS PLAYING

    IP_total_bow = {}

    # Dictionaries: Key - Date/Time    Value - Number of tweets
    IP_total_tweets = {}
    IP_tweets_per_hour = {}

    IP_neg_polarity = {}
    IP_neu_polarity = {}
    IP_pos_polarity = {}
    IP_com_polarity = {}

    IP_yes_cognition_old = {}
    IP_no_cognition_old = {}

    IP_yes_cognition_new = {}
    IP_no_cognition_new = {}

    IP_happy_sentiment_old = {}
    IP_sad_sentiment_old = {}
    IP_general_sentiment_old = {}

    IP_happy_sentiment_new = {}
    IP_sad_sentiment_new = {}
    IP_general_sentiment_new = {}


    # DICTS FOR TEAM IS NOT PLAYING

    NP_total_bow = {}

    # Dictionaries: Key - Date/Time    Value - Number of tweets
    NP_total_tweets = {}
    NP_tweets_per_hour = {}

    NP_neg_polarity = {}
    NP_neu_polarity = {}
    NP_pos_polarity = {}
    NP_com_polarity = {}

    NP_yes_cognition_old = {}
    NP_no_cognition_old = {}

    NP_yes_cognition_new = {}
    NP_no_cognition_new = {}

    NP_happy_sentiment_old = {}
    NP_sad_sentiment_old = {}
    NP_general_sentiment_old = {}

    NP_happy_sentiment_new = {}
    NP_sad_sentiment_new = {}
    NP_general_sentiment_new = {}

    object_generator = storage_client.list_blobs(BUCKET_NAME, prefix=team_name, delimiter=None)

    for status_log_bucket_name in object_generator:

        # TODO: Delete me
        bucket_time = time.time()
        if bucket_time - t0 > 10:
            break

        client2 = storage.Client()
        BUCKET_NAME_TWO = client2.bucket(BUCKET_NAME)
        blob = BUCKET_NAME_TWO.blob(status_log_bucket_name.name)

        with blob.open('rt') as f:
            for line in f:
                try:
                    raw_tweet = json.loads(line)

                    # TODO: Delete me
                    start_parse_time = time.time()
                    print('\nACCESS TIME  - ', start_parse_time - bucket_time, '\n')

                    tweet = Tweet(raw_tweet)

                    text = clean_tweet(tweet.all_text)
                    
                    print('Text:', text, '\n\n')

                    # Extracts time, date, and language
                    date = tweet.created_at_string[0:10]
                    print('Date:', date, '\n')
                    hour = int(tweet.created_at_string[11:13])
                    print('Hour:', hour, '\n')
                    lang = tweet.lang
                    print('Lang:', lang, '\n')

                    # Increment combined total number of tweets
                    increment_dict(date, COM_total_tweets)
                    print('COM_total_tweets:', COM_total_tweets, '\n')
                    increment_dict(date, ALL_TEAMS_total_tweets)
                    print('ALL_TEAMS_total_tweets:', ALL_TEAMS_total_tweets, '\n')
                    increment_dict(hour, COM_tweets_per_hour)
                    print('COM_tweets_per_hour:', COM_tweets_per_hour, '\n')

                    # Increment combined total BoW
                    words = text.split()
                    for word in words:
                        increment_dict(word, COM_total_bow)
                        increment_dict(word, ALL_TEAMS_total_bow)
                        
                    print('COM_total_bow:', COM_total_bow, '\n')
                    print('ALL_TEAMS_total_bow:', ALL_TEAMS_total_bow, '\n')
                    

                    # Gets coordinates from tweet
                    coordinates = tweet.geo_coordinates
                    if coordinates is not None and coordinates['latitude'] is not None and coordinates['longitute'] is not None:
                        tweet_lat = coordinates['latitude']
                        tweet_long = coordinates['longitute']

                        tweet_coords = (tweet_lat, tweet_long)

                        team_lat = team_lats[team_abbrev]
                        team_long = team_longs[team_abbrev]

                        team_coords = (team_lat, team_long)

                        # Distance between the tweet and stadium (meters)
                        distance = geopy.distance.geodesic(tweet_coords, team_coords).km * 1000
                        print('Tweet coords:', tweet_coords, '\n')
                        print('Team coords:', team_coords, '\n')
                        print('Distance:', distance, '\n')

                        # Updates average distance
                        if (COM_average_distance != 0):
                            COM_average_distance = COM_average_distance * (COM_total_tweets[date] - 1) / COM_total_tweets[date] + distance / COM_total_tweets[date]
                        else:
                            COM_average_distance = distance
                            
                        print('COM average distance:', COM_average_distance, '\n')



                        # !! TWEET PARSING !!

                        # If the team is playing
                        
                        
                        # TODO: UNCOMMENT!
                        # if date in dates and hour >= 10 and lang == 'en':

                        # Increment total number of tweets
                        increment_dict(date, IP_total_tweets)
                        increment_dict(hour, IP_tweets_per_hour)

                        # Increment total BoW
                        for word in words:
                            increment_dict(word, IP_total_bow)

                        # Updates average distance
                        if (IP_average_distance != 0):
                            IP_average_distance = IP_average_distance * (IP_total_tweets[date] - 1) / IP_total_tweets[date] + distance / IP_total_tweets[date]
                        else:
                            IP_average_distance = distance


                        # !! POLARITY !!

                        scores = sia.polarity_scores(text)  # Create a dict of scores

                        # Loop the dict of scores and increment the dicts if necessary
                        for key in scores:
                            if key == 'neg':
                                if scores[key] == 0.0:
                                    increment_dict(date, IP_neg_polarity)
                            if key == 'neu':
                                if scores[key] == 0.0:
                                    increment_dict(date, IP_neu_polarity)
                            if key == 'pos':
                                if scores[key] == 0.0:
                                    increment_dict(date, IP_pos_polarity)
                            if key == 'compound':
                                if scores[key] == 0.0:
                                    increment_dict(date, IP_com_polarity)


                        # !! COGNITION !!

                        tokens = nltk.word_tokenize(text)  # Create a list of tokens
                        tokens_tagged = nltk.pos_tag(tokens, tagset='universal')

                        pos_tokens = ''

                        # Loop the dict of tokens
                        for token in tokens_tagged:
                            pos_tokens += token[1] + ' '

                        sentence = str(pos_tokens)  # 'Whoever is happy will make others happy too'
                        trigrams = ngrams(sentence.split(), 3)
                        quadgrams = ngrams(sentence.split(), 4)

                        trigram_list = []
                        for item in trigrams:
                            trigram_list.append(item[0])
                            trigram_list.append(item[1])
                            trigram_list.append(item[2])

                        quadgram_list = []
                        for item in quadgrams:
                            quadgram_list.append(item[0])
                            quadgram_list.append(item[1])
                            quadgram_list.append(item[2])
                            quadgram_list.append(item[3])

                        # If in old list, add to dict
                        if trigram_list in super_list_old or quadgram_list in super_list_old:
                            increment_dict(date, IP_yes_cognition_old)
                        else:
                            increment_dict(date, IP_no_cognition_old)

                        # If in new list, add to dict
                        if trigram_list in super_list_new or quadgram_list in super_list_new:
                            increment_dict(date, IP_yes_cognition_new)
                        else:
                            increment_dict(date, IP_no_cognition_new)


                        # !! SENTIMENT !!

                        # If in old happy emoticons, add to dict
                        for emoticon in emoticons_happy_old:
                            if text.lower().find(emoticon) != -1:
                                increment_dict(date, IP_happy_sentiment_old)
                                increment_dict(date, IP_general_sentiment_old)
                                break;

                        # If in old sad emoticons, add to dict
                        for emoticon in emoticons_sad_old:
                            if text.lower().find(emoticon) != -1:
                                increment_dict(date, IP_sad_sentiment_old)
                                increment_dict(date, IP_general_sentiment_old)
                                break;

                        # If in new happy emoticons, add to dict
                        for emoticon in emoticons_happy_new:
                            if text.lower().find(emoticon) != -1:
                                increment_dict(date, IP_happy_sentiment_new)
                                increment_dict(date, IP_general_sentiment_new)
                                break;

                        # If in new sad emoticons, add to dict
                        for emoticon in emoticons_sad_new:
                            if text.lower().find(emoticon) != -1:
                                increment_dict(date, IP_sad_sentiment_new)
                                increment_dict(date, IP_general_sentiment_new)
                                break;

                        # TODO: Delete me
                        end_parse_time = time.time()
                        print('\nTWEET PARSING TIME  - ', end_parse_time - start_parse_time, '\n\n')
                        
                        time.sleep(1)



                        # If the team is NOT playing
                        # TODO: UNCOMMENT
#                         else:

#                             # Increment total number of tweets
#                             increment_dict(date, NP_total_tweets)
#                             increment_dict(hour, NP_tweets_per_hour)

#                             # Increment total BoW
#                             for word in words:
#                                 increment_dict(word, NP_total_bow)

#                             # Updates average distance
#                             if (IP_average_distance != 0):
#                                 IP_average_distance = IP_average_distance * (IP_total_tweets[date] - 1) / IP_total_tweets[date] + distance / IP_total_tweets[date]
#                             else:
#                                 IP_average_distance = distance


#                             # !! POLARITY !!

#                             scores = sia.polarity_scores(text)  # Create a dict of scores

#                             # Loop the dict of scores and increment the dicts if necessary
#                             for key in scores:
#                                 if key == 'neg':
#                                     if scores[key] == 0.0:
#                                         increment_dict(date, NP_neg_polarity)
#                                 if key == 'neu':
#                                     if scores[key] == 0.0:
#                                         increment_dict(date, NP_neu_polarity)
#                                 if key == 'pos':
#                                     if scores[key] == 0.0:
#                                         increment_dict(date, NP_pos_polarity)
#                                 if key == 'compound':
#                                     if scores[key] == 0.0:
#                                         increment_dict(date, NP_com_polarity)


#                             # !! COGNITION !!

#                             tokens = nltk.word_tokenize(text)  # Create a list of tokens
#                             tokens_tagged = nltk.pos_tag(tokens, tagset='universal')

#                             pos_tokens = ''

#                             # Loop the dict of tokens
#                             for token in tokens_tagged:
#                                 pos_tokens += token[1] + ' '

#                             sentence = str(pos_tokens)  # 'Whoever is happy will make others happy too'
#                             trigrams = ngrams(sentence.split(), 3)
#                             quadgrams = ngrams(sentence.split(), 4)

#                             trigram_list = []
#                             for item in trigrams:
#                                 trigram_list.append(item[0])
#                                 trigram_list.append(item[1])
#                                 trigram_list.append(item[2])

#                             quadgram_list = []
#                             for item in quadgrams:
#                                 quadgram_list.append(item[0])
#                                 quadgram_list.append(item[1])
#                                 quadgram_list.append(item[2])
#                                 quadgram_list.append(item[3])

#                             # If in old list, add to dict
#                             if trigram_list in super_list_old or quadgram_list in super_list_old:
#                                 increment_dict(date, NP_yes_cognition_old)
#                             else:
#                                 increment_dict(date, NP_no_cognition_old)

#                             # If in new list, add to dict
#                             if trigram_list in super_list_new or quadgram_list in super_list_new:
#                                 increment_dict(date, NP_yes_cognition_new)
#                             else:
#                                 increment_dict(date, NP_no_cognition_new)


#                             # !! SENTIMENT !!

#                             # If in old happy emoticons, add to dict
#                             for emoticon in emoticons_happy_old:
#                                 if text.lower().find(emoticon) != -1:
#                                     increment_dict(date, NP_happy_sentiment_old)
#                                     increment_dict(date, NP_general_sentiment_old)
#                                     break;

#                             # If in old sad emoticons, add to dict
#                             for emoticon in emoticons_sad_old:
#                                 if text.lower().find(emoticon) != -1:
#                                     increment_dict(date, NP_sad_sentiment_old)
#                                     increment_dict(date, NP_general_sentiment_old)
#                                     break;

#                             # If in new happy emoticons, add to dict
#                             for emoticon in emoticons_happy_new:
#                                 if text.lower().find(emoticon) != -1:
#                                     increment_dict(date, NP_happy_sentiment_new)
#                                     increment_dict(date, NP_general_sentiment_new)
#                                     break;

#                             # If in new sad emoticons, add to dict
#                             for emoticon in emoticons_sad_new:
#                                 if text.lower().find(emoticon) != -1:
#                                     increment_dict(date, NP_sad_sentiment_new)
#                                     increment_dict(date, NP_general_sentiment_new)
#                                     break;

#                             # TODO: Delete me
#                             end_parse_time = time.time()
#                             print('\nTWEET PARSING TIME  - ', end_parse_time - start_parse_time, '\n')

                except:
                    pass


    # Write to combined average distance + total BoW/tweet text files
    with open(team_abbrev + '_COM_average_distance.txt', 'w') as new_file:
        new_file.write(COM_average_distance)
    write_dict_to_file(team_abbrev + '_COM_total_bow.txt', COM_total_bow)
    write_dict_to_file(team_abbrev + '_COM_total_tweets.txt', COM_total_tweets)
    write_dict_to_file(team_abbrev + '_COM_tweets_per_hour.txt', COM_tweets_per_hour)


    # TEAM IS PLAYING

    # Write to average distance + total BoW/tweet text files
    with open(team_abbrev + '_IP_average_distance.txt', 'w') as new_file:
        new_file.write(IP_average_distance)
    write_dict_to_file(team_abbrev + '_IP_total_bow.txt', total_bow)
    write_dict_to_file(team_abbrev + '_IP_total_tweets.txt', total_tweets)
    write_dict_to_file(team_abbrev + '_IP_tweets_per_hour.txt', IP_tweets_per_hour)

    # Write to polarity text files
    write_dict_to_file(team_abbrev + '_IP_negative.txt', neg_polarity)
    write_dict_to_file(team_abbrev + '_IP_neutral.txt', neu_polarity)
    write_dict_to_file(team_abbrev + '_IP_positive.txt', pos_polarity)
    write_dict_to_file(team_abbrev + '_IP_compound.txt', com_polarity)

    # Write to cognition text files old
    write_dict_to_file(team_abbrev + '_IP_yes_old.txt', yes_cognition_old)
    write_dict_to_file(team_abbrev + '_IP_no_old.txt', no_cognition_old)

    # Write to cognition text files new
    write_dict_to_file(team_abbrev + '_IP_yes_new.txt', yes_cognition_new)
    write_dict_to_file(team_abbrev + '_IP_no_new.txt', no_cognition_new)

    # Write to sentiment text files old
    write_dict_to_file(team_abbrev + '_IP_happy_old.txt', happy_sentiment_old)
    write_dict_to_file(team_abbrev + '_IP_sad_old.txt', sad_sentiment_old)
    write_dict_to_file(team_abbrev + '_IP_general_old.txt', general_sentiment_old)

    # Write to sentiment text files new
    write_dict_to_file(team_abbrev + '_IP_happy_new.txt', happy_sentiment_new)
    write_dict_to_file(team_abbrev + '_IP_sad_new.txt', sad_sentiment_new)
    write_dict_to_file(team_abbrev + '_IP_general_new.txt', general_sentiment_new)


    # TEAM NOT PLAYING:

    # Write to average distance text file
    with open(team_abbrev + '_NP_average_distance.txt', 'w') as new_file:
        new_file.write(NP_average_distance)

    # Write to total BoW/tweet text files
    write_dict_to_file(team_abbrev + '_NP_total_bow.txt', NP_total_bow)
    write_dict_to_file(team_abbrev + '_NP_total_tweets.txt', NP_total_tweets)
    write_dict_to_file(team_abbrev + '_NP_tweets_per_hour.txt', NP_tweets_per_hour)

    # Write to polarity text files
    write_dict_to_file(team_abbrev + '_NP_negative.txt', NP_neg_polarity)
    write_dict_to_file(team_abbrev + '_NP_neutral.txt', NP_neu_polarity)
    write_dict_to_file(team_abbrev + '_NP_positive.txt', NP_pos_polarity)
    write_dict_to_file(team_abbrev + '_NP_compound.txt', NP_com_polarity)

    # Write to cognition text files old
    write_dict_to_file(team_abbrev + '_NP_yes_old.txt', NP_yes_cognition_old)
    write_dict_to_file(team_abbrev + '_NP_no_old.txt', NP_no_cognition_old)

    # Write to cognition text files new
    write_dict_to_file(team_abbrev + '_NP_yes_new.txt', NP_yes_cognition_new)
    write_dict_to_file(team_abbrev + '_NP_no_new.txt', NP_no_cognition_new)

    # Write to sentiment text files old
    write_dict_to_file(team_abbrev + '_NP_happy_old.txt', NP_happy_sentiment_old)
    write_dict_to_file(team_abbrev + '_NP_sad_old.txt', NP_sad_sentiment_old)
    write_dict_to_file(team_abbrev + '_NP_general_old.txt', NP_general_sentiment_old)

    # Write to sentiment text files new
    write_dict_to_file(team_abbrev + '_NP_happy_new.txt', NP_happy_sentiment_new)
    write_dict_to_file(team_abbrev + '_NP_sad_new.txt', NP_sad_sentiment_new)
    write_dict_to_file(team_abbrev + '_NP_general_new.txt', NP_general_sentiment_new)

    t1 = time.time()

    print(team_name, 'DONE: Runtime  - ', display_time(t1 - t0))


# ALL TEAMS

# Write to total BoW/tweet text files
write_dict_to_file('ALL_TEAMS_total_bow.txt', ALL_TEAMS_total_bow)
write_dict_to_file('ALL_TEAMS_total_tweets.txt', ALL_TEAMS_total_tweets)


t_final = time.time()

print('\nTOTAL RUNTIME  - ', display_time(t_final - t0), '\n')
print('El Fin')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # remove old style retweet text 'RT'
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags
    tweet = re.sub(r'#', '', tweet)
    
    return tweet

intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
)

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(round(value), name))
    return ', '.join(result[:granularity])


t0 = time.time()


for team_name in team_names:
    
    dates = []
    
    team_abbrev = team_abbrevs[team_names.index(team_name)]

    for i in range(16, 20):
        with open(('Data/Team_WL/' + str(team_abbrev) + str(i) + '.csv'), newline='', encoding='utf-8-sig') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in reader:
                date = row[0]
                if (date.startswith('d')):
                    continue
                if not date.startswith('1'):
                    date = '0' + date
                if not date[5] == '/':
                    date = date[:3] + '0' + date[3:]
                date = date[6:] + '-' + date[:2] + '-' + date[3:-5]
                dates.append(date)

    # Dictionaries: Key - Date    Value - Number of tweets
    neg_polarity_dict = {}
    neu_polarity_dict = {}
    pos_polarity_dict = {}
    com_polarity_dict = {}
    
    yes_cognition_dict = {}
    no_cognition_dict = {}
    
    happy_sentiment_dict = {}
    sad_sentiment_dict = {}
    general_sentiment_dict = {}
 
    object_generator = storage_client.list_blobs(BUCKET_NAME, prefix=team_name, delimiter=None)

    for status_log_bucket_name in object_generator:
        client2 = storage.Client()
        BUCKET_NAME_TWO = client2.bucket(BUCKET_NAME)
        blob = BUCKET_NAME_TWO.blob(status_log_bucket_name.name)

        with blob.open('rt') as f:
            for line in f:
                try:
                    tweet_dict = json.loads(line)

                    tweet = Tweet(tweet_dict)

                    text = clean_tweets(tweet.all_text)

                    # Extracts date and verifies its a day the Sox play
                    date = tweet.created_at_string[0:10]

                    if date in dates:
                        if date[:4] == '2019':

                            # POLARITY

                            scores = sia.polarity_scores(text) # Create a dict of scores

                            # Loop the dict of scores
                            for key in scores:
                                if key == 'neg':
                                    if scores[key] == 0.0:
                                        if date in neg_polarity_dict.keys():
                                            neg_polarity_dict[date] += 1
                                        else:
                                            neg_polarity_dict[date] = 1
                                if key == 'neu':
                                    if scores[key] == 0.0:
                                        if date in neu_polarity_dict.keys():
                                            neu_polarity_dict[date] += 1
                                        else:
                                            neu_polarity_dict[date] = 1
                                if key == 'pos':
                                    if scores[key] == 0.0:
                                        if date in pos_polarity_dict.keys():
                                            pos_polarity_dict[date] += 1
                                        else:
                                            pos_polarity_dict[date] = 1
                                if key == 'compound':
                                    if scores[key] == 0.0:
                                        if date in com_polarity_dict.keys():
                                            com_polarity_dict[date] += 1
                                        else:
                                            com_polarity_dict[date] = 1


                            # COGNITION

                            tokens = nltk.word_tokenize(text) # Create a list of tokens
                            tokens_tagged = nltk.pos_tag(tokens, tagset='universal')

                            pos_tokens = ''

                            # Loop the dict of tokens
                            for token in tokens_tagged:
                                pos_tokens += token[1] + ' '

                            sentence = str(pos_tokens)#'Whoever is happy will make others happy too'
                            trigrams = ngrams(sentence.split(), 3)
                            quadgrams = ngrams(sentence.split(), 4)

                            trigram_list = []
                            for item in trigrams:
                                trigram_list.append(item[0])
                                trigram_list.append(item[1])
                                trigram_list.append(item[2])

                            quadgram_list = []
                            for item in quadgrams:
                                quadgram_list.append(item[0])
                                quadgram_list.append(item[1])
                                quadgram_list.append(item[2])
                                quadgram_list.append(item[3])

                            if trigram_list in super_list or quadgram_list in super_list:
                                if date in yes_cognition_dict.keys():
                                    yes_cognition_dict[date] += 1
                                else:
                                    yes_cognition_dict[date] = 1
                            else:
                                if date in no_cognition_dict.keys():
                                    no_cognition_dict[date] += 1
                                else:
                                    no_cognition_dict[date] = 1


                            # SENTIMENT        

                            for emoticon in emoticons_happy:
                                if text.lower().find(emoticon) != -1:
                                  # There's a happy emoticon: Adds to the dict
                                  if date in happy_sentiment_dict.keys():
                                    happy_sentiment_dict[date] += 1
                                  else:
                                    happy_sentiment_dict[date] = 1
                                  if date in general_sentiment_dict.keys():
                                    general_sentiment_dict[date] += 1
                                  else:
                                    general_sentiment_dict[date] = 1
                                  break;

                            for emoticon in emoticons_sad:
                                if text.lower().find(emoticon) != -1:
                                  # There's a sad emoticon: Adds to the dict
                                  if date in sad_sentiment_dict.keys():
                                    sad_sentiment_dict[date] += 1
                                  else:
                                    sad_sentiment_dict[date] = 1
                                  if date in general_sentiment_dict.keys():
                                    general_sentiment_dict[date] += 1
                                  else:
                                    general_sentiment_dict[date] = 1
                                  break;
                    
                except:
                    pass
    
    
#     # Write to polarity text files
#     with open(team_abbrev + '_negative.txt', 'w') as new_file: 
#         for key, value in neg_polarity_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

#     with open(team_abbrev + '_neutral.txt', 'w') as new_file: 
#         for key, value in neu_polarity_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

#     with open(team_abbrev + '_positive.txt', 'w') as new_file: 
#         for key, value in pos_polarity_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

#     with open(team_abbrev + '_compound.txt', 'w') as new_file: 
#         for key, value in com_polarity_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))
            
            
#     # Write to cognition text files
#     with open(team_abbrev + '_yes.txt', 'w') as new_file: 
#         for key, value in yes_cognition_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

#     with open(team_abbrev + '_no.txt', 'w') as new_file: 
#         for key, value in no_cognition_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))
            
            
#     # Write to sentiment text files
#     with open(team_abbrev + '_happy.txt', 'w') as new_file: 
#         for key, value in happy_sentiment_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

#     with open(team_abbrev + '_sad.txt', 'w') as new_file: 
#         for key, value in sad_sentiment_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

#     with open(team_abbrev + '_general.txt', 'w') as new_file: 
#         for key, value in general_sentiment_dict.items(): 
#             new_file.write('%s : %d\n' % (key, value))

    # Write to polarity text files
    with open(team_abbrev + '19_negative.txt', 'w') as new_file: 
        for key, value in neg_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open(team_abbrev + '19_neutral.txt', 'w') as new_file: 
        for key, value in neu_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open(team_abbrev + '19_positive.txt', 'w') as new_file: 
        for key, value in pos_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open(team_abbrev + '19_compound.txt', 'w') as new_file: 
        for key, value in com_polarity_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))
            
            
    # Write to cognition text files
    with open(team_abbrev + '19_yes.txt', 'w') as new_file: 
        for key, value in yes_cognition_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open(team_abbrev + '19_no.txt', 'w') as new_file: 
        for key, value in no_cognition_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))
            
            
    # Write to sentiment text files
    with open(team_abbrev + '19_happy.txt', 'w') as new_file: 
        for key, value in happy_sentiment_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open(team_abbrev + '19_sad.txt', 'w') as new_file: 
        for key, value in sad_sentiment_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))

    with open(team_abbrev + '19_general.txt', 'w') as new_file: 
        for key, value in general_sentiment_dict.items(): 
            new_file.write('%s : %d\n' % (key, value))
            
    t1 = time.time()
            
    print(team_name, 'runtime  - ', display_time(t1-t0))
                    

t_final = time.time()
        
print('\nTOTAL RUNTIME  - ', display_time(t_final-t0), '\n')
print('El Fin')

KeyboardInterrupt: 